In [1]:
!pip install -q bitsandbytes transformers peft accelerate diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [2]:
import os
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm.auto import tqdm
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler
import numpy as np
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig
from safetensors.torch import save_file, load_file
from accelerate import Accelerator, load_checkpoint_in_model
import torch.nn as nn

2025-12-18 13:10:29.354639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766063429.536920      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766063429.590956      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
def load_catvton(unet, weight_path):    
    if not os.path.exists(weight_path):
        raise FileNotFoundError("File bobot tidak ditemukan!")

    state_dict = load_file(weight_path)
    
    # Ambil semua layer Attention 1 dari Model UNet
    model_layers = [m for name, m in unet.named_modules() if name.endswith("attn1")]
    
    file_idx = 0
    loaded_count = 0
    
    for layer in model_layers:
        key_q = f"{file_idx}.to_q.weight"
        
        if key_q not in state_dict:
            break
            
        # --- LOGIKA PENCOCOKAN ---
        file_dim = state_dict[key_q].shape[0]
        model_dim = layer.to_q.out_features
        
        if file_dim == model_dim:
            layer.to_q.weight.data = state_dict[f"{file_idx}.to_q.weight"]
            layer.to_k.weight.data = state_dict[f"{file_idx}.to_k.weight"]
            layer.to_v.weight.data = state_dict[f"{file_idx}.to_v.weight"]
            layer.to_out[0].weight.data = state_dict[f"{file_idx}.to_out.0.weight"]
            layer.to_out[0].bias.data = state_dict[f"{file_idx}.to_out.0.bias"]
            
            file_idx += 1
            loaded_count += 1
        else:
            pass

    print(f"Selesai")

In [4]:
unet_temp = UNet2DConditionModel.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    subfolder="unet",
    torch_dtype=torch.float16
)

# Masukkin bobot base catvton
catvton_weight_path = "/kaggle/input/catvton/model.safetensors"
print("Injecting CatVTON weights")
load_catvton(unet_temp, catvton_weight_path)

save_dir = "/kaggle/working/catvton_base"
print(f"Menyimpan model ke: {save_dir}")
unet_temp.save_pretrained(save_dir)

del unet_temp
torch.cuda.empty_cache()

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

An error occurred while trying to fetch runwayml/stable-diffusion-inpainting: runwayml/stable-diffusion-inpainting does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


unet/diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Injecting CatVTON weights
Selesai
Menyimpan model ke: /kaggle/working/catvton_base


In [5]:
CONFIG = {
    "data_root": "/kaggle/input/high-resolution-viton-zalando-dataset", 
    "output_dir": "/kaggle/working/catvton_qlora",
    "resolution": (512, 384),
    "batch_size": 8,
    "grad_accum_steps": 16,
    "num_epochs": 2,
    "lr": 2e-5,
    "lora_rank": 16,
    "lora_alpha": 32
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class VitonSpatialDataset(Dataset):
    def __init__(self, root_dir, size=(512, 384)):
        self.size = size
        self.image_dir = os.path.join(root_dir, "train", "image")
        self.cloth_dir = os.path.join(root_dir, "train", "cloth")
        self.mask_dir = os.path.join(root_dir, "train", "image-parse-v3")
        
        if not os.path.exists(self.image_dir):
            self.image_dir = os.path.join(root_dir, "image")
            self.cloth_dir = os.path.join(root_dir, "cloth")
            self.mask_dir = os.path.join(root_dir, "image-parse-v3")
            
        self.files = sorted(os.listdir(self.image_dir)) if os.path.exists(self.image_dir) else []
        self.transform = transforms.Compose([transforms.Resize(size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])
        self.mask_transform = transforms.Compose([transforms.Resize(size, interpolation=transforms.InterpolationMode.NEAREST), transforms.ToTensor()])

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        try:
            f = self.files[idx]
            img = Image.open(os.path.join(self.image_dir, f)).convert("RGB")
            cloth = Image.open(os.path.join(self.cloth_dir, f)).convert("RGB")
            
            mask_name = f.replace(".jpg", ".png")
            if not os.path.exists(os.path.join(self.mask_dir, mask_name)): mask_name = f
            mask = Image.open(os.path.join(self.mask_dir, mask_name)).convert("L")
            
            img_t = self.transform(img)
            cloth_t = self.transform(cloth)
            mask_t = self.mask_transform(mask) 
            mask_t = (mask_t > 0.5).float()
            
            masked_img_t = img_t * (1 - mask_t)
            
            # Gabungin gambar secara vertikal (SPATIAL CONCATENATION)
            concat_image = torch.cat([img_t, cloth_t], dim=1)
            concat_masked = torch.cat([masked_img_t, cloth_t], dim=1)
            
            zeros_mask = torch.zeros_like(mask_t)
            concat_mask = torch.cat([mask_t, zeros_mask], dim=1)
            
            return {
                "image": concat_image, 
                "mask": concat_mask, 
                "masked_image": concat_masked
            }
        except Exception as e:
            return self.__getitem__((idx + 1) % len(self))

def save_lora_weights(model, save_path, lora_config):
    os.makedirs(save_path, exist_ok=True)
    state_dict = model.state_dict()
    lora_state_dict = {}
    for name, param in model.named_parameters():
        if param.requires_grad or "lora" in name:
            lora_state_dict[name] = param.cpu()
            
    output_file = os.path.join(save_path, "adapter_model.safetensors")
    save_file(lora_state_dict, output_file)
    lora_config.save_pretrained(save_path)
    print(f"LoRA Saved: {output_file} ({len(lora_state_dict)} tensors)")
    
# SETUP MODEL
accelerator = Accelerator(
    gradient_accumulation_steps=CONFIG["grad_accum_steps"],
    mixed_precision="fp16" 
)

os.makedirs(CONFIG["output_dir"], exist_ok=True)
model_id = "runwayml/stable-diffusion-inpainting"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True,
)

vae = AutoencoderKL.from_pretrained(model_id, subfolder="vae").to(accelerator.device)
scheduler = DDPMScheduler.from_pretrained(model_id, subfolder="scheduler")

unet = UNet2DConditionModel.from_pretrained(
    "/kaggle/working/catvton_base",
    quantization_config=bnb_config,
    device_map="auto"
)
unet.enable_gradient_checkpointing()
vae.requires_grad_(False)
unet.train()

lora_config = LoraConfig(
    r=CONFIG["lora_rank"],
    lora_alpha=CONFIG["lora_alpha"],
    target_modules=["to_k", "to_q", "to_v", "to_out.0"],
    init_lora_weights="gaussian" 
)
unet.add_adapter(lora_config)

optimizer = bnb.optim.AdamW(filter(lambda p: p.requires_grad, unet.parameters()), lr=CONFIG["lr"])

dataset = VitonSpatialDataset(CONFIG["data_root"], size=CONFIG["resolution"])
dataloader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=2)

unet, optimizer, dataloader = accelerator.prepare(unet, optimizer, dataloader)

print("Mulai Training dengan Accelerate")

for epoch in range(CONFIG["num_epochs"]):
    unet.train()
    epoch_loss = 0.0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}", disable=not accelerator.is_local_main_process)
    
    for step, batch in enumerate(progress_bar):
        with accelerator.accumulate(unet):
            img = batch["image"]
            mask_raw = batch["mask"]
            masked_img = batch["masked_image"]

            with torch.no_grad():
                latents = vae.encode(img).latent_dist.sample() * 0.18215
                masked_latents = vae.encode(masked_img).latent_dist.sample() * 0.18215
                mask = torch.nn.functional.interpolate(mask_raw, size=latents.shape[-2:], mode="nearest")
            
            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, 1000, (latents.shape[0],), device=accelerator.device)
            noisy_latents = scheduler.add_noise(latents, noise, timesteps)
            
            # Forward
            unet_input = torch.cat([noisy_latents, mask, masked_latents], dim=1)
            dummy_emb = torch.zeros((img.shape[0], 1, 768), device=accelerator.device)
            
            noise_pred = unet(unet_input, timesteps, encoder_hidden_states=dummy_emb).sample
            loss = F.mse_loss(noise_pred, noise)
            
            # Backward
            accelerator.backward(loss)
            
            optimizer.step()
            optimizer.zero_grad()
            
            loss_value = loss.detach().item()
            epoch_loss += loss_value
            progress_bar.set_postfix({"loss": loss_value})

    unwrapped_unet = accelerator.unwrap_model(unet)
    save_path = os.path.join(CONFIG["output_dir"], f"catvton_lora_epoch_{epoch}")
    save_lora_weights(unet, save_path, lora_config)    
    print(f"Epoch {epoch} Selesai. Avg Loss: {epoch_loss/len(dataloader):.4f}")

print("Selesai!")

config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

An error occurred while trying to fetch runwayml/stable-diffusion-inpainting: runwayml/stable-diffusion-inpainting does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


vae/diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

Mulai Training dengan Accelerate


Epoch 0:   0%|          | 0/1456 [00:00<?, ?it/s]

LoRA Saved: /kaggle/working/catvton_qlora/catvton_lora_epoch_0/adapter_model.safetensors (256 tensors)
Epoch 0 Selesai. Avg Loss: 0.0049


Epoch 1:   0%|          | 0/1456 [00:00<?, ?it/s]

LoRA Saved: /kaggle/working/catvton_qlora/catvton_lora_epoch_1/adapter_model.safetensors (256 tensors)
Epoch 1 Selesai. Avg Loss: 0.0045
Selesai!
